## Examen  práctico 2020-2021

__ETL, MDS__

__Valentina Díaz Torres__


<p style='text-align: justify;'>El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio.</p>

<p style='text-align: justify;'>En este caso, vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio.</p>



|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

### Carga del contexto Spark

In [1]:
# Librerias e inicializacion y carga del contexto spark

import re
from pyspark.sql import functions as f
from pyspark.sql import Row
import numpy as np
import pandas as pd
from pyspark import SparkContext
pd.options.display.max_columns = None 
pd.options.display.max_rows = None 
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)
from pyspark.sql import *
from pyspark.sql import SQLContext, functions as F, Row
from pyspark.sql.types import *
from pyspark.sql.types import StringType, IntegerType
import findspark
findspark.init()
import pyspark 
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Test").getOrCreate()
from pyspark.sql.functions import desc
from pyspark.sql.functions import col
pd.set_option('max_columns',None) #para que no me corte columnas ni filas
pd.set_option('max_rows',None)
from pyspark.sql.functions import upper, col
from pyspark.sql.functions import isnan, when, count

### Carga de datos y parseado

En primer lugar, se cargan los dos archivos proporcionados, en formato excel y en csv. No obstante, se va a trabajar con el archivo en formato csv.

In [55]:
#carga del archivo excel 
pdf = pd.read_excel('BDpracticafinal2021.xlsx')
df_e = spark.createDataFrame(pdf.astype(str))

In [3]:
#carga del archivo csv
df = spark.read.format('csv').option('header',True).option('multiLine', True).option("sep", ";").load('BDpracticafinalCSV.csv')

__Exploración de los datos__

In [4]:
df.columns

['Order',
 'MS Zoning',
 'Lot Frontage',
 'Lot Area',
 'Land Contour',
 'Land Slope',
 'Overall Qual',
 'Year Built',
 'Year Remod/Add',
 'Mas Vnr Type',
 'Exter Qual',
 'Bsmt Cond',
 'Total Bsmt SF',
 'Heating QC',
 'Central Air',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'Kitchen Qual',
 'TotRms AbvGrd',
 'Garage Cars',
 'Garage Area',
 'Garage Cond',
 'Pool Area',
 'Pool QC',
 'Mo Sold',
 'Yr Sold',
 'SalePrice']

<p style='text-align: justify;'>

Tras observar el formato en el que se encuentran las columnas, se ha visto necesario unir las palabras, formando una sola. Esto se ha hecho mediante una fución que sustituye el espacio entre palabra y palabra por un guión bajo ("_").
</p>

In [4]:
etiqueta = df.columns #almacenar todas las columnas dentro del objeto etiqueta
#A continuación, se crea una función para que sustituya por _ los espacios entre las palabras, a lo largo de todas las columans
def sepGuiones(columnas):
    for i in range(len(etiqueta)):
        columnas[i] = str(etiqueta[i].replace(" ","_"))#se pasa a string porque la función replace lo necesita
    return columnas

In [5]:
cadena = sepGuiones(etiqueta) #se aplica la función que se ha definido antes a las columnas y se le pone el nombre de cadena
#un bucle para renombrar todas las columnas 
for i in range(len(df.columns)):
    df = df.withColumnRenamed(df.columns[i] , cadena[i])


<p style='text-align: justify;'>Tras ello, se explorarn otros aspectos tales como el tipo de cada columna o el número de columnas y filas que tiene el dataset.</p>

In [7]:
df.dtypes #tipo de cada variable

[('Order', 'string'),
 ('MS_Zoning', 'string'),
 ('Lot_Frontage', 'string'),
 ('Lot_Area', 'string'),
 ('Land_Contour', 'string'),
 ('Land_Slope', 'string'),
 ('Overall_Qual', 'string'),
 ('Year_Built', 'string'),
 ('Year_Remod/Add', 'string'),
 ('Mas_Vnr_Type', 'string'),
 ('Exter_Qual', 'string'),
 ('Bsmt_Cond', 'string'),
 ('Total_Bsmt_SF', 'string'),
 ('Heating_QC', 'string'),
 ('Central_Air', 'string'),
 ('Full_Bath', 'string'),
 ('Half_Bath', 'string'),
 ('Bedroom_AbvGr', 'string'),
 ('Kitchen_AbvGr', 'string'),
 ('Kitchen_Qual', 'string'),
 ('TotRms_AbvGrd', 'string'),
 ('Garage_Cars', 'string'),
 ('Garage_Area', 'string'),
 ('Garage_Cond', 'string'),
 ('Pool_Area', 'string'),
 ('Pool_QC', 'string'),
 ('Mo_Sold', 'string'),
 ('Yr_Sold', 'string'),
 ('SalePrice', 'string')]

<p style='text-align: justify;'>Todas las variables son de tipo string, por lo que, una vez que se han leído todas las consultas que se van a tener que realizar, se decide pasar dos de las variables a tipo double.</p>

In [6]:
#cambiar el tipo de las columnas que me hacen falta
df = df.withColumn("SalePrice",df["SalePrice"].cast(DoubleType()))

In [7]:
#cambiar el tipo de las columnas que me hacen falta
df = df.withColumn("Total_Bsmt_SF",df["Total_Bsmt_SF"].cast(DoubleType()))

In [8]:
len(df.columns) #numero de columnas : 29

29

In [9]:
df.count() #numero de filas: 2936

2936

<p style='text-align: justify;'>
Además, teniendo en cuenta que interesa solo trabajar con las zonas de RH residencial alta densidad, RL residencial baja densidad y RM residencial media densidad, en primer lugar, se observan qué zonas se encuentran en el dataset y luego se filtra, para trabajar solo con las tres que interesa.</p>
<p style='text-align: justify;'>
Es necesario resaltar también, que en la zona RL, existían algunos valores que se detectaban como distintas categorías, porque no estaban escritos de una forma homogénea, como la mayoría (RL), sino que tenían alguna de sus letras en minúsculas. Este problema se ha resuelto transformando todas las categorías a mayúscula, por lo que se han detectado como una sola, RL.</p>

In [12]:
#cambio a mayúscula
df = df.withColumn("MS_Zoning",F.upper(F.col("MS_Zoning")))
#comprobación
df.groupby("MS_Zoning").count().show()

+---------+-----+
|MS_Zoning|count|
+---------+-----+
|       RH|   27|
|       FV|  139|
|        C|   25|
|        A|    2|
|       RL| 2279|
|        I|    2|
|       RM|  462|
+---------+-----+



In [13]:
#Hacer el filtro por las tres categorías
df = df.filter((df["MS_Zoning"] == "RL") | (df["MS_Zoning"] == "RM") | (df["MS_Zoning"] == "RH"))

__Tratamiento de valores nulos__

<p style='text-align: justify;'>Respecto a los valores nulos, estos son detectado mediante una función que recorre todas las columnas, buscando posibles valores nulos y muestra una lista con el total de nulos por variable.</p>

In [14]:
#detección de nulos
etiqueta = df.columns
def muestra():
    for i in range(len(df.columns)):
        print(etiqueta[i] + ": ",df.filter(df[etiqueta[i]].isNull()).count())   

In [15]:
muestra()#observar que valores son nulos

Order:  0
MS_Zoning:  0
Lot_Frontage:  469
Lot_Area:  0
Land_Contour:  0
Land_Slope:  0
Overall_Qual:  0
Year_Built:  0
Year_Remod/Add:  0
Mas_Vnr_Type:  16
Exter_Qual:  0
Bsmt_Cond:  0
Total_Bsmt_SF:  0
Heating_QC:  0
Central_Air:  0
Full_Bath:  0
Half_Bath:  0
Bedroom_AbvGr:  0
Kitchen_AbvGr:  0
Kitchen_Qual:  0
TotRms_AbvGrd:  0
Garage_Cars:  1
Garage_Area:  1
Garage_Cond:  2
Pool_Area:  0
Pool_QC:  0
Mo_Sold:  0
Yr_Sold:  0
SalePrice:  0


<p style='text-align: justify;'>Se han detectado valores nulos en algunas de las variables. En primer lugar, se ha decido que en las columnas Lot Fontage y Mas Vnr Type, que son las que más valores nulos tienen, y por tanto, eliminarlos podría producir una pérdida de información significativa, van a ser sustituidos por 0. En el caso de los valores de las columnas de Garage Cars, Area y Cond, los valores nulos son eliminados, porque son muy pocos.</p>

In [16]:
#sustituir por 0 los valores nulos de las columnas que mas tienen 
df =  df.fillna({'Lot_Frontage':'0', 'Mas_Vnr_Type':'0'}) 
#eliminar los valores nulos de las columnas restantes, ya que son muy pocos
df = df.na.drop(subset = ["Garage_Cars","Garage_Area", "Garage_Cond"])

In [17]:
muestra() #comprobar que se han cambiado

Order:  0
MS_Zoning:  0
Lot_Frontage:  0
Lot_Area:  0
Land_Contour:  0
Land_Slope:  0
Overall_Qual:  0
Year_Built:  0
Year_Remod/Add:  0
Mas_Vnr_Type:  0
Exter_Qual:  0
Bsmt_Cond:  0
Total_Bsmt_SF:  0
Heating_QC:  0
Central_Air:  0
Full_Bath:  0
Half_Bath:  0
Bedroom_AbvGr:  0
Kitchen_AbvGr:  0
Kitchen_Qual:  0
TotRms_AbvGrd:  0
Garage_Cars:  0
Garage_Area:  0
Garage_Cond:  0
Pool_Area:  0
Pool_QC:  0
Mo_Sold:  0
Yr_Sold:  0
SalePrice:  0


__Detección de valores duplicados__

<p style='text-align: justify;'>Por otro lado, los valores nulos son identificados mediante una función. Pero de momento, estos no son tratados. El tratamiento se realizará en próximas preguntas.</p>

In [18]:
#comprobar duplicados
dup= df.join(
    df.groupBy(df.columns).agg((f.count("*")>1).cast("int").alias("Duplicate_indicator")),
    on=df.columns,
    how="inner"
)
dup.filter(dup["Duplicate_indicator"] == "1").count() # hay 12 valores duplicados

12

### Pregunta 2. ¿Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? ¿Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

<p style='text-align: justify;'>Como ya se comentó anteriormente, los valores duplicados no se trataron con el fin de poder hacerlo en esta pregunta.</p>

<p style='text-align: justify;'>En primer lugar, se pretende responder a la primera cuestión, que consiste en conocer el número de viviendas que existe. Para ello, se ha tomado como referente, el ID de cada una de las viviendas, la columna "Order", que asigna un número a cada una. </p>

<p style='text-align: justify;'>Para conocer cuántas viviendas hay y si se repite alguna, se realiza un conteo de las filas del dataset, una vez que se ha realizado la limpieza. Este valor es de 2766. A continuación, se realiza la función distinct para que cuente cuántas casas, no repetidas, hay, en este caso, aparecen 2760. Esto indicaría que habría 6 valores duplicados, como ya se comprobó antes.</p>

In [19]:
df.count()

2766

In [20]:
viviendas = df.groupBy("Order").sum().distinct().collect()
len(viviendas)

2760

<p style='text-align: justify;'>El número de viviendas entonces sería de 2760. Para detectar qué viviendas son las que tienen valores nulos se realiza un filtro, en el que se le pone como condición que tengan una frecuencia superior a 1.</p>

In [21]:
df1=df.groupBy('Order').count().filter("count > 1")
df1.drop('count').show()

+-----+
|Order|
+-----+
| 2898|
| 2930|
| 2929|
| 2900|
| 2899|
| 2901|
+-----+



<p style='text-align: justify;'>Los número mostrados, corresponden a las seis casas duplicadas. Respondiendo a la cuestión de si sería lógico o no, se ha considerado que no, ya que debería existir un solo registro por cada casa.</p>

<p style='text-align: justify;'>El dataset recoge las características de cada vivienda y por ejemplo, no tendría sentido que una casa estuviese en más de una zona o tuviese dos características diferentes de número de garajes. Por ello, se ha decidido eliminarlos, con el fin de poder realizar una análisis por vivienda sin datos que puedan afectar.</p>

In [22]:
df = df.dropDuplicates(['Order']) #eliminar duplicados

### Pregunta 3.¿Podrías decirme el total de inmuebles y el precio medio de cada zona?

<p style='text-align: justify;'>Para responder a esta cuestión, se ha realizado, por un lado, el precio de venta medio por cada zona y por otro, el nuúmero de casas total por cada zona. Luego se ha unido todo en una tabla para mejor visualización.</p>

In [23]:
media_precio = df.groupBy("MS_Zoning").agg(f.round(f.avg("SalePrice"),2).alias("media_precio"))

In [24]:
total_Inmueble_msz = df.groupBy("MS_Zoning").agg(f.round(f.count('Order'),2).alias('total_inmuebles'))

In [25]:
union1 = total_Inmueble_msz.join(media_precio, on="MS_Zoning", how = "inner")
union1.show()

+---------+---------------+------------+
|MS_Zoning|total_inmuebles|media_precio|
+---------+---------------+------------+
|       RH|             27|   136419.78|
|       RL|           2273|   191283.25|
|       RM|            460|   126656.73|
+---------+---------------+------------+



<p style='text-align: justify;'>Los resultados obtenidos muestran que hay un mayor número de viviendas en la zona RL y el menor número se encuentra en la RH, también que los precios medios más altos se encuentran en las viviendas de RL, en la zona con más viviendas.</p>

### Pregunta 4. Media de Total Bsmt_SF por cada década de construcción calculada a partir de Year Built.

<p style='text-align: justify;'>Para llevar a cabo esta consulta, en primer lugar, ha sido necesario crear una columna llamada década, que recoge la década a la que pertenece cada uno de los años de construcción registrados. Una vez que se ha podido hacer uso de esta varaible, se ha calculado la media de la variable Total_Bsmt_SF (Superficie del sótano en pies cuadrados), por década de construcción.</p>

In [26]:
#crear una columna llamada decada
df = df.withColumn("decade", (f.floor(f.col("Year_Built")/10)*10).cast("int"))

In [27]:
media_bsmt_sf = df.groupBy("decade").agg(f.round(f.avg("Total_Bsmt_SF"),2).alias("media_Bsmt_Sf"))
#ordenar los datos según la década de menor a mayor
tabla_ordenada= media_bsmt_sf.orderBy("decade", ascending = True)
tabla_ordenada.show()

+------+-------------+
|decade|media_Bsmt_Sf|
+------+-------------+
|  1870|        761.0|
|  1880|        836.0|
|  1890|       885.58|
|  1900|       720.14|
|  1910|       783.85|
|  1920|        833.8|
|  1930|       769.92|
|  1940|       706.93|
|  1950|        968.3|
|  1960|      1082.67|
|  1970|       952.72|
|  1980|       1086.0|
|  1990|      1138.81|
|  2000|      1348.47|
|  2010|       1539.0|
+------+-------------+



<p style='text-align: justify;'>Los resultados obtenidos indican que las décadas correspondientes a las viviendas más grandes son las de 1960,1980,1990,2000 y 2010.</p>

### Pregunta 5.¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

<p style='text-align: justify;'>Para conocer la década con las viviendas mejor acondicionadas para el frío, primero se realiza filtran aquellas viviendas de la categoría excelente(Ex) de la variable Heatin QC y luego se agrupa por década. Así se obtiene una tabla que compara las décadas, con el número de viviendas que tienen el mejor acondicionamiento.</p>

In [28]:
mejorCondicion_dec = df.filter(df.Heating_QC == "Ex").groupBy("decade").pivot("Heating_QC").count()
#ordenar de mayor a menor por número de casas con condición excelnte
mejorCondicion_ord = mejorCondicion_dec.orderBy("Ex",ascending= False)
mejorCondicion_ord.show()

+------+---+
|decade| Ex|
+------+---+
|  2000|638|
|  1990|233|
|  1950| 98|
|  1960| 88|
|  1970| 68|
|  1940| 56|
|  1920| 50|
|  1930| 40|
|  1910| 38|
|  1980| 28|
|  1900| 12|
|  1890|  4|
|  1880|  2|
|  2010|  2|
+------+---+



<p style='text-align: justify;'>Los resultados obtenidos, aparecen ordenados, mostrando en los primeros valrores de la tabla, aquellas décadas que tienen un mayor número de viviendas con mejor acondicionamiento. Estas son 2000 y 1990. Lo que podría significar que a partir de 1990, las casas empezaron a estar mucho mejor acondicionadas para el frío. En 2000 ya hay una diferencia mucho mayor. </p>

### Pregunta 6.¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

<p style='text-align: justify;'>Para dar respuesta a esta cuestión, primero es necesario realizar una transformación de piés a metros cuadrados, para poder hacer la comparación de aquellas viviendas que tienen un valor más elevado por metro cuadrado. Posteriormente, se filtran los datos, para quedarse solo con los del año 2009. Seguidamente, se agrupan por el identificador de la vivienda, para conocer cuáles son estas 10 viviendas y por la columna de metros cuadrados. Por último se muestran también los precios totales de dichas viviendas.</p>

In [29]:
#pasar  de pies a metros cuadraddos
df = df.withColumn("Lot_Area_m2", f.round(df["Lot_Area"]*0.09,2))

In [30]:
datos_2009 = df.filter(df["Yr_Sold"] == 2009) 
precio_m2 = datos_2009.groupBy(["Order","Lot_Area_m2"]).agg(f.sum("SalePrice").alias("precio"))
#ordenar por precio el resultado me mayor a menor
precio_m2.orderBy('precio', ascending= False).show(10)

+-----+-----------+--------+
|Order|Lot_Area_m2|  precio|
+-----+-----------+--------+
|  433|    1232.37|610000.0|
|  434|    1250.19|582933.0|
|  424|    1388.79|555000.0|
|  457|    1335.24|552000.0|
|  367|    1568.07|501837.0|
|  423|    1216.62|485000.0|
|  514|    1001.52|468000.0|
|  422|    1283.13|462000.0|
|  432|    1213.02|451950.0|
|  449|    1374.66|445000.0|
+-----+-----------+--------+
only showing top 10 rows



<p style='text-align: justify;'>
Las viviendas se muestran ordenadas de mayor a menor precio. Así se obtiene que la que tiene un mayor precio es la número 433. Sabiendo esto, se podrían comprobar sus características en las demás variables.</p>


### Pregunta 7. Media anual por zonas del precio de venta y metros cuadrados.

<p style='text-align: justify;'>
En esta cuestión se ha realizado una agrupación por año de venta y por zona y luego se ha calculado el precio medio para cada una de las zonas y año y la media de metros cuadrados.</p>

In [31]:
mediaPrecio_zona_m2 = df.groupBy(["Yr_Sold","MS_Zoning"]).agg((f.round(f.avg("SalePrice"),2).alias("Precio")),
(f.round(f.avg("Lot_Area_m2"),2).alias("m²")))
mediaPrecio_zona_m2.show()

+-------+---------+---------+-------+
|Yr_Sold|MS_Zoning|   Precio|     m²|
+-------+---------+---------+-------+
|   2006|       RL|194350.64|1011.18|
|   2010|       RM|119681.71| 567.79|
|   2008|       RL|189885.59|1014.92|
|   2006|       RM|127502.64| 557.05|
|   2009|       RM|118759.87| 535.29|
|   2007|       RM|128713.99| 603.68|
|   2008|       RH|150333.33|  742.2|
|   2007|       RL|193399.98|1026.57|
|   2010|       RH| 126280.0|  729.5|
|   2008|       RM|133117.57| 603.16|
|   2007|       RH| 147250.0| 630.97|
|   2010|       RL|184978.42| 979.76|
|   2009|       RL|190455.55| 970.87|
|   2009|       RH| 116206.8| 626.92|
|   2006|       RH|140316.67| 638.96|
+-------+---------+---------+-------+



### Pregunta 8.¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

<p style='text-align: justify;'>Para obtener el total recaudado, es decir, el total de precio de venta, por cada casa, según el tipo de revestimiento, piedra o ladrillo, se ha filtrado y agrupado por cada uno de los tipos de revestimiento que interesaba. No obstante, como existen dos tipos de revestimiento de ladrillo, se ha el total recaudado por la venta de las viviendas de ambos tipos, ya que no quedaba especificado cuál era el revestimiento requerido.</p>

In [32]:
#tipos de revestimiento
df.groupby("Mas_Vnr_Type").count().show() 

+------------+-----+
|Mas_Vnr_Type|count|
+------------+-----+
|        None| 1641|
|      BrkCmn|   25|
|           0|   16|
|       Stone|  229|
|     BrkFace|  848|
|      CBlock|    1|
+------------+-----+



In [33]:
#recaudación total por las viviendas de revestimiento de piedra
recaudacion_piedra = df.filter(df.Mas_Vnr_Type == "Stone").groupBy("SalePrice").count()
recaudacion_piedra.agg(f.round((f.sum("SalePrice")),-6).alias("Piedra: Precio")).show()

+--------------+
|Piedra: Precio|
+--------------+
|         5.2E7|
+--------------+



In [34]:
#recaudación total por las viviendas de revestimiento del primer tipo de ladrillo (Ladrillo normal)
recaudacion_ladrillo1 = df.filter(df.Mas_Vnr_Type == "BrkCmn").groupBy("SalePrice").count()
recaudacion_ladrillo1.agg((f.sum("SalePrice")).alias("Ladrillo Normal: Precio")).show()

+-----------------------+
|Ladrillo Normal: Precio|
+-----------------------+
|              3504975.0|
+-----------------------+



In [35]:
#recaudación total por las viviendas de revestimiento del segundo tipo de ladrillo (Ladrillo visto)
recaudacion_ladrillo2 = df.filter(df.Mas_Vnr_Type == "BrkFace").groupBy("SalePrice").count()
recaudacion_ladrillo2.agg(f.round(f.sum("SalePrice"),-6).alias("Ladrillo visto: Precio")).show()

+----------------------+
|Ladrillo visto: Precio|
+----------------------+
|                1.06E8|
+----------------------+



Los resultados muestran:

* Que el total de recaudación por las casas con revestimiento de piedra ha sido de 51 millones aproximadamente, por la venta de 230 viviendas. 

* En el caso del revestimiento de ladrillo normal, existen 25 viviendas y se ha recaudado un total de 3504975$.

* En el caso del ladrillo visto, se ha obtenido la mayor recaudación, existen 850 viviendas con este revestimiento y se ha obtenido aproximadamente 106 millones de Dólares.

Por lo tanto, se podría concluir que el revestimiento de ladrillo visto es el más frecuente, el que se encuentra en el mayor número de viviendas y por eso supone la mayor recaudación del total.

### Pregunta 9.¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

<p style='text-align: justify;'>Este análisis se realiza de forma separada, primero se filtran aquellas casas que tienen 2 cocinas y dos o más plazas de garaje y por otro lado las que tienen una cocina y una sola plaza de garaje. Además, para comparar ambos grupos, se obtienen los principales estadísticos (que se aplican a las únicas variables convertidas a numéricas) y los cuartiles del precio de dichos conjuntos.</p>

In [36]:
#filtrado por la primera condici´´on
condicion_1 = df.filter((df["Kitchen_AbvGr"] == 2) & (df["Garage_Cars"] >= 2 ))
c1_precio = condicion_1.groupBy(["Kitchen_AbvGr","Garage_Cars"]).agg(f.sum("SalePrice").alias("precio"))
c1_precio.show()

+-------------+-----------+---------+
|Kitchen_AbvGr|Garage_Cars|   precio|
+-------------+-----------+---------+
|            2|          3|1387500.0|
|            2|          2|9229688.0|
|            2|          4|1863500.0|
+-------------+-----------+---------+



In [37]:
#principales estadísticos
condicion_1.describe().show() 

+-------+------------------+------------------+------------------+
|summary|         SalePrice|            decade|       Lot_Area_m2|
+-------+------------------+------------------+------------------+
|  count|                86|                86|                86|
|   mean|145124.27906976745|1955.1162790697674| 890.6222093023257|
| stddev| 40555.86827593908|26.334686857339822|333.37986058796315|
|    min|           81400.0|              1880|            544.32|
|    max|          269500.0|              2000|           2940.12|
+-------+------------------+------------------+------------------+



In [38]:
#cuartiles 
c1_precio.approxQuantile("precio",[0.25, 0.5, 0.75], 0) #el 0 significa error relativo, se usa 0 para tener el
#mayor accuracy posible

[1863500.0, 9229688.0, 9229688.0]

In [39]:
#filtrado por la segunda condicion
condicion_2 = df.filter((df["Kitchen_AbvGr"] == 1) & (df["Garage_Cars"] == 1 ))
c2_precio = condicion_2.groupBy(["Kitchen_AbvGr","Garage_Cars"]).agg(f.round(f.sum("SalePrice"),-6).alias("precio"))
c2_precio.show()

+-------------+-----------+------+
|Kitchen_AbvGr|Garage_Cars|precio|
+-------------+-----------+------+
|            1|          1| 9.7E7|
+-------------+-----------+------+



In [40]:
#principales estadísticos de la segunda condición
condicion_2.describe().show() 

+-------+------------------+------------------+-----------------+
|summary|         SalePrice|            decade|      Lot_Area_m2|
+-------+------------------+------------------+-----------------+
|  count|               754|               754|              754|
|   mean| 128079.7148541114|1945.0928381962865|760.7302519893893|
| stddev|30301.638905480977| 19.84145329086487|304.9795976143738|
|    min|           35000.0|              1880|           132.93|
|    max|          330000.0|              2000|          2492.73|
+-------+------------------+------------------+-----------------+



In [41]:
#cuartiles 
c2_precio.approxQuantile("precio",[0.25, 0.5, 0.75], 0)

[97000000.0, 97000000.0, 97000000.0]

<p style='text-align: justify;'>Analizando los resultados obtenidos se puede observar que:</p>

<p style='text-align: justify;'>

• En el primer grupo hay 86 viviendas con dichas condiciones, que el precio medio es 145124.279, el maximo 269500.0 y la vivienda más barata es de 81400.0, la desviación típica es de 40555.86. Son viviendas con 2 cocinas y 2, 3 o 4 plazas de garaje. Los cuartiles de precio, y por tanto los rangos de precio menor, medio y mayor son: 1863500, 9229688, 9229688 Dólares.
</p>
<p style='text-align: justify;'>
• En el segundo grupo, se puede observar que hay 754 viviendas y por tanto, este tipo de viviendas es el más frecuente. Además, el precio medio es 128079.7148 Dólares, por lo que son más baratas, el máximo 330000 Dólares y la vivienda más barata es de 35000 Dólares. La desviación típica es de 30301.6389, lo que significa que hay menos variación en los precios que en las anteriores. Los cuartiles de precios son: 96572105, 96572105, 96572105 Dólares</p>

### Pregunta 10. Estudiar la relación entre el precio y el número de Garajes. 

<p style='text-align: justify;'>Para estudiar la relación que tienen el precio de venta y el número de garajes de la vivienda, se podrían plantear muchas opciones. En este caso, se han calculado los cuartiles del precio y se ha analizado la relación entre el númeor de garajes y el precio en cada uno de estos cuartiles. También, se ha calculado el precio medio de las viviendas en cada cuartil y el número total de viviendas, para poder hacer posteriores comparaciones.
</p>

In [42]:
#precio segmentado por cuartiles
df.approxQuantile("SalePrice",[0.25, 0.5, 0.75], 0) 

[129000.0, 159000.0, 212000.0]

In [43]:
# Se filtra por rangos de precio, inferior o igual que el primer cuartil, el segundo mayor que ese y el tercero el resto
precio_cuartil1 = df.filter(df["SalePrice"] <= 129000.0)
precio_cuartil2 = df.filter((df["SalePrice"] > 129000.0) & (df["SalePrice"] <= 159000.0))
precio_cuartil3 = df.filter(df["SalePrice"] > 159000.0)

In [44]:
a1 = precio_cuartil1.groupBy("Garage_Cars").agg((f.round(f.avg("SalePrice"),2).alias("precio medio")), (f.count("Order").alias("nº de viviendas")))
a1.show()

+-----------+------------+---------------+
|Garage_Cars|precio medio|nº de viviendas|
+-----------+------------+---------------+
|          3|   107571.43|              7|
|          0|    93004.73|            111|
|          5|    126500.0|              1|
|          1|   108280.23|            411|
|          4|    123000.0|              1|
|          2|   110465.89|            170|
+-----------+------------+---------------+



In [45]:
#precio medio de las viviendas del primer cuartil
a1.agg(f.round(f.avg("precio medio"),2).alias("Precio Medio de las Viviendas (1er Cuartil)")).show()

+-------------------------------------------+
|Precio Medio de las Viviendas (1er Cuartil)|
+-------------------------------------------+
|                                  111470.38|
+-------------------------------------------+



In [46]:
#todas las viviendas que se encuentran en el primer cuartil
a1.agg(f.round(f.sum("nº de viviendas"),2).alias("Nº de Viviendas Totales (1er Cuartil)")).show()

+-------------------------------------+
|Nº de Viviendas Totales (1er Cuartil)|
+-------------------------------------+
|                                  701|
+-------------------------------------+



In [47]:
a2 = precio_cuartil2.groupBy("Garage_Cars").agg((f.round(f.avg("SalePrice"),2).alias("precio medio")), (f.count("Order").alias("nº de viviendas")))
a2.show()

+-----------+------------+---------------+
|Garage_Cars|precio medio|nº de viviendas|
+-----------+------------+---------------+
|          3|   146691.67|             12|
|          0|    139662.5|             28|
|          1|   141356.44|            279|
|          4|    134500.0|              2|
|          2|   144820.58|            366|
+-----------+------------+---------------+



In [48]:
#precio medio de las viviendas del segundo cuartil
a2.agg(f.round(f.avg("precio medio"),2).alias("Precio Medio de las Viviendas (2º Cuartil)")).show()

+------------------------------------------+
|Precio Medio de las Viviendas (2º Cuartil)|
+------------------------------------------+
|                                 141406.24|
+------------------------------------------+



In [49]:
#todas las viviendas que se encuentran en el segundo cuartil
a2.agg(f.round(f.sum("nº de viviendas"),2).alias("Nº de Viviendas Totales (2º Cuartil)")).show()

+------------------------------------+
|Nº de Viviendas Totales (2º Cuartil)|
+------------------------------------+
|                                 687|
+------------------------------------+



In [50]:
a3 = precio_cuartil3.groupBy("Garage_Cars").agg((f.round(f.avg("SalePrice"),2).alias("precio medio")), (f.count("Order").alias("Nº de viviendas")))
a3.show()

+-----------+------------+---------------+
|Garage_Cars|precio medio|Nº de viviendas|
+-----------+------------+---------------+
|          3|    322327.0|            336|
|          0|    187300.0|              8|
|          1|   187661.42|             73|
|          4|    251383.0|             13|
|          2|   209101.59|            942|
+-----------+------------+---------------+



In [51]:
#precio medio de las viviendas del tercer cuartil
a3.agg(f.round(f.avg("precio medio"),2).alias("Precio Medio de las Viviendas (3er Cuartil)")).show()

+-------------------------------------------+
|Precio Medio de las Viviendas (3er Cuartil)|
+-------------------------------------------+
|                                   231554.6|
+-------------------------------------------+



In [52]:
#Todas las viviendas que se encuentran en el tercer cuartil
a3.agg(f.round(f.sum("nº de viviendas"),2).alias("Número de viviendas totales (3er Cuartil)")).show()

+-----------------------------------------+
|Número de viviendas totales (3er Cuartil)|
+-----------------------------------------+
|                                     1372|
+-----------------------------------------+



El resultado obtenido ha sido:

<p style='text-align: justify;'>• En el primer grupo, correspondiente a las viviendas del primer cuartil de precios y por tanto las casas más baratas, se encuentran 701 casas y el precio medio de ellas es de 111470.38. El número de plazas de garaje es de 0 a 5. La mayoría de viviendas tienen 1 o 2 garajes. Solo hay una vivienda con 5 plazas de garaje y el precio medio de este es 126500 dolares. </p>

<p style='text-align: justify;'>
• En el segundo grupo, se encuentran 687 viviendas y el precio medio de estas es de 141406.24 dolares. Aquí no existen casas con 5 garajes, la mayoría de ellas tienen 1 o dos plazas de garaje también. </p>

<p style='text-align: justify;'>
• En el tercer cuartil, el de las viviendas más caras, el número de casas es de 1378 y el precio medio de estas es  231467.06 dolares. En este caso, la mayoría de viviendas tienen 2 o tres plazas de garaje, a diferencia de los dos grupos anteriores.</p>
<p style='text-align: justify;'>

Por lo tanto, por lo general, se puede decir que hay más casas con un precio más elevado, correspondiente al tercer cuartil y que estas, las viviendas más caras, suelen tener entre dos y tres plazas de garajes, mientras que las de precio bajo y medio suelen tener entre una y dos plazas.</p>

### Pregunta 11. Las 10 viviendas con mejores servicios y mejor precio.

Para plantear este ejercicio tendríamos dos posibles opciones:

<p style='text-align: justify;'>1) Plantear el filtrado a través de la columna Overall_Qual, que como se explica arriba es la que proporciona la información acerca del grado de calidad de materiales y acabado de la vivienda, por lo que se entiende que cuanto mayor sea el nº de esta columna, la calidad general de la vivienda será mejor.</p>

<p style='text-align: justify;'>2) Plantear el filtrado con cada una de las columnas que determinan la calidad de la vivienda, es decir:</p>

<p style='text-align: justify;'>
        •        Exter_Qual, Bsmt_Cond, Kitchen_Qual, Garage_Cond, Pool_QC y Heating_QC.</p>
    
<p style='text-align: justify;'> Con esta elección, sería necesario interpretar a juicio propio qué formaría una casa con los mejores servicios, ya que por ejemplo, el hecho de tener piscina o no, según para quién, puede ser algo importante a la hora de tener una vivienda completa. De hecho, observando el dataset, se puede comprobar cómo hay un número significante de casas que no cuentan con piscina (2747/2760), por lo que si se hiciese un filtro propio priorizando en aquellas viviendas que la tienen, se obtendrían valores totalmente distintos a los que se podrían conseguir si no se tuviesen las piscinas en cuenta.</p>
  
<p style='text-align: justify;'>Debido a la ambigüedad que puede generar la elección de la ejecución 2), he optado por realizar la primera de la siguiente manera:  </p>

In [10]:
#Realizo un filtrado de aquellas viviendas que tengan en la columna Overall_Qual el valor igual a 10 (Las mejores)
mejoresViviendas = df.filter((df["Overall_Qual"] == 10))

#Una vez almacenadas en la variable mejoresViviendas, las agrupo por Overall_Qual y Order, y mediante SalePrice 
#obtengo el precio de la vivienda según mi agrupación en una nueva columna llamada Precio vivienda
mejorV = mejoresViviendas.groupby(["Overall_Qual",'Order']).agg(f.sum("SalePrice").alias("Precio vivienda"))

#Ordeno descendentemente por la columna Precio vivienda, y así obtengo la vivienda más barata 
#que posee una media de calidad de componentes excelente.
top10viviendas = mejorV.orderBy('Precio vivienda',ascending = True).limit(10)

#Muestro el resultado final
top10viviendas.show()

+------------+-----+---------------+
|Overall_Qual|Order|Precio vivienda|
+------------+-----+---------------+
|          10| 1499|       160000.0|
|          10| 2181|       183850.0|
|          10| 2182|       184750.0|
|          10|  524|       310000.0|
|          10| 1321|       325000.0|
|          10|  421|       337500.0|
|          10| 1178|       345000.0|
|          10| 1053|       385000.0|
|          10|  448|       386250.0|
|          10| 2276|       392000.0|
+------------+-----+---------------+



La tabla anterior muestra las viviendas que tienen una mejor calidad-precio según el grado de calidad del material con el que están hechas. La vivienda 1499 por ejemplo, tiene el mejor precio de todas, teniendo una excelente calidad en la media de sus materiales. 

Otra opción sería, evaluar las variables una por una. Es decir, si se considera que una vivienda tiene mejor por la piscina, por el acondicionamiento del frío, por la calidad de la cocina o por la del garaje, entre otras. Así, habría que evaluar por separado cada una de ellas y realizar un filtrado para conocer cuáles son estas viviendas. A continuación se han realizado diferentes agrupaciones según las varaibles nombradas anteriormente y se han mostrado en tablas cuáles son estas viviendas. 

In [11]:
calidad = df.groupBy('Pool_QC').agg(f.count("Pool_QC").alias("Calidad"))
calidad1 = df.groupBy('Exter_Qual').agg(f.count("Exter_Qual").alias("Calidad"))
calidad2= df.groupBy('Heating_QC').agg(f.count("Heating_QC").alias("Calidad"))
calidad3 = df.groupBy('Kitchen_Qual').agg(f.count("Kitchen_Qual").alias("Calidad"))
calidad4 = df.groupBy('Garage_Cond').agg(f.count("Garage_Cond").alias("Calidad"))
calidad.show()
calidad1.show()
calidad2.show()
calidad3.show()
calidad4.show()

+-------+-------+
|Pool_QC|Calidad|
+-------+-------+
|     Gd|      4|
|     Sp|   2923|
|     Ex|      4|
|     Fa|      2|
|     TA|      3|
+-------+-------+

+----------+-------+
|Exter_Qual|Calidad|
+----------+-------+
|        Gd|    992|
|        Ex|    107|
|        Fa|     35|
|        TA|   1802|
+----------+-------+

+----------+-------+
|Heating_QC|Calidad|
+----------+-------+
|        Gd|    477|
|        Po|      3|
|        Ex|   1500|
|        Fa|     92|
|        TA|    864|
+----------+-------+

+------------+-------+
|Kitchen_Qual|Calidad|
+------------+-------+
|          Gd|   1163|
|          Po|      1|
|          Ex|    205|
|          Fa|     70|
|          TA|   1497|
+------------+-------+

+-----------+-------+
|Garage_Cond|Calidad|
+-----------+-------+
|         Gd|     15|
|       null|      0|
|         Po|     14|
|         Sg|    157|
|         Ex|      3|
|         Fa|     74|
|         TA|   2671|
+-----------+-------+



Sin embargo, se se comprueban cuáles son estas viviendas, y las características que tienen con respecto a otras variables, se observa que si tiene puntuación excelente en una columna, como por ejemplo condición excelente de piscina, no tiene excelencia en todas las demás. Es por eso, que se concluye diciendo que no se puede hacer un filtrado general de las viviendas que contienen excelente en todo, sino que habría que estudiarlo según las preferencias y lo que busca el cliente.

In [12]:
mejorCondicion_dec1 = df.filter(df.Pool_QC == "Ex")
mejorCondicion_dec1.toPandas().head()

,Order,MS_Zoning,Lot_Frontage,Lot_Area,Land_Contour,Land_Slope,Overall_Qual,Year_Built,Year_Remod/Add,Mas_Vnr_Type,Exter_Qual,Bsmt_Cond,Total_Bsmt_SF,Heating_QC,Central_Air,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Garage_Cars,Garage_Area,Garage_Cond,Pool_Area,Pool_QC,Mo_Sold,Yr_Sold,SalePrice
0,1064,RL,106,12720,HLS,Mod,10,2003,2003,Stone,Ex,TA,2535.0,Ex,Y,1,1,1,1,Ex,7,3,789,TA,144,Ex,2,2008,615000.0
1,1761,RL,160,15623,Lvl,Gtl,10,1996,1996,None,Gd,TA,2396.0,Ex,Y,3,1,4,1,Ex,10,3,813,TA,555,Ex,7,2007,745000.0
2,2231,RL,70,18044,HLS,Gtl,8,1986,1986,None,Gd,TA,279.0,Gd,Y,2,1,2,1,Gd,6,2,691,Gd,228,Ex,8,2007,315000.0
3,2571,RL,174,25419,Lvl,Gtl,8,1918,1990,None,Gd,TA,1360.0,Gd,Y,2,0,4,1,Gd,8,2,795,TA,512,Ex,3,2006,235000.0


In [13]:
mejoresViviendas = df.filter((df["Exter_Qual"] == "Ex") & (df["Bsmt_Cond"] == "Ex") & (df["Heating_QC"] == "Ex") & (df["Kitchen_Qual"] == "Ex")  & (df["Garage_Cond"] == "TA") & (df["Pool_QC"] == "Sp"))
#mejorV = mejoresViviendas.groupby(['Order']).agg(f.sum("SalePrice").alias("precio"))
#mejorV.orderBy('precio',ascending = True).show(10)
mejoresViviendas.toPandas().head(15)

,Order,MS_Zoning,Lot_Frontage,Lot_Area,Land_Contour,Land_Slope,Overall_Qual,Year_Built,Year_Remod/Add,Mas_Vnr_Type,Exter_Qual,Bsmt_Cond,Total_Bsmt_SF,Heating_QC,Central_Air,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,Kitchen_Qual,TotRms_AbvGrd,Garage_Cars,Garage_Area,Garage_Cond,Pool_Area,Pool_QC,Mo_Sold,Yr_Sold,SalePrice
0,1184,RL,103,16280,Lvl,Gtl,8,1976,2007,None,Ex,Ex,1426.0,Ex,Y,3,0,3,1,Ex,6,2,550,TA,0,Sp,5,2008,290000.0
